This document demonstrates the making, training, saving, loading, and usage of a sklearn-compliant CGCNN model.

In [1]:
import os
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'/home/zulissi/software/adamwr')
import numpy as np
import cgcnn
#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


## Load the dataset as mongo docs

In [2]:
import random
import pickle

#Load a selection of documents
docs = pickle.load(open('/pylon5/ch5fq5p/zulissi/CO_docs.pkl','rb'))
random.seed(42)
random.shuffle(docs)
docs = [doc for doc in docs if -3<doc['energy']<1.0]
docs = docs[:10]

## Get the size of the features from the data transformer, to be used in setting up the net model

In [3]:
from torch.utils.data import Dataset, DataLoader
import mongo
from cgcnn.data import StructureData, ListDataset, StructureDataTransformer
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler


SDT = StructureDataTransformer(atom_init_loc='../atom_init.json',
                              max_num_nbr=12,
                               step=0.2,
                              radius=1,
                              use_tag=True,
                              use_fixed_info=False,
                              use_distance=True)

SDT_out = SDT.transform(docs)

structures = SDT_out[0]

#Settings necessary to build the model (since they are size of vectors as inputs)
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

import multiprocess as mp
from sklearn.model_selection import ShuffleSplit

SDT_out = SDT.transform(docs)

with mp.Pool(16) as pool:
    SDT_list = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))

100%|██████████| 10/10 [00:14<00:00,  1.46s/it]


In [4]:
with open('distance_all_docs.pkl','wb') as fhandle:
    pickle.dump(SDT_list,fhandle)

## CGCNN model with skorch to make it sklearn compliant

In [5]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it
from cgcnn.data import collate_pool
from skorch import NeuralNetRegressor
from cgcnn.model import CrystalGraphConvNet
import torch
from cgcnn.data import MergeDataset
import skorch.callbacks.base


cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training
class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()


## Example converting all the documents up front

In [6]:
#Make the target list
target_list = np.array([doc['energy'] for doc in docs]).reshape(-1,1)

## Shuffle and Split

In [7]:
from sklearn.model_selection import train_test_split

SDT_training, SDT_test, target_training, target_test = train_test_split(SDT_list, target_list, test_size=0.2)


## Fit the model

In [10]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler
from adamw import AdamW
from cosine_scheduler import CosineLRWithRestarts

train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)

# warm restart scheduling from https://arxiv.org/pdf/1711.05101.pdf
LR_schedule = LRScheduler(CosineLRWithRestarts, batch_size=214, epoch_size=len(SDT_training), restart_period=10, t_mult=1.2)

# Tune variables with SIGOPT
# Note to self(qingyanz) - Focus on other issues instead
net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    batch_size=214,                 # tune this
    module__classification=False,
    lr=0.0056,                      #tune this
    max_epochs=100,                 #tune this
    module__atom_fea_len=46,        #tune this
    module__h_fea_len=83,            #tune this
    module__n_conv=8,               #tune this
    module__n_h=6,                  #tune this
    # module__visualization=False,    #Switch to per-atom energy mode
    optimizer=AdamW,                #could be adamw, adam, sgd, etc
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
#     criterion=torch.nn.MSELoss,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)



In [11]:
# PyTorch model with a Scorch interface, so we can use it as sklearn object
net.initialize()
net.fit(SDT_training,target_training)

Re-initializing module because the following parameters were re-set: atom_fea_len, classification, h_fea_len, n_conv, n_h, nbr_fea_len, orig_atom_fea_len.
Re-initializing optimizer because the following parameters were re-set: .


RuntimeError: Epoch size and batch size used in the training loop and while initializing scheduler should be the same.

Add a hook to save the per-atom visualization each time the net is called

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

net.module_.visualization_layer.register_forward_hook(get_activation('visualization_layer'))


Save 5 atoms objects with the calculated per-atom energy as the "charge" attribute.  

In [ ]:
index=0
for sdt,doc in zip(SDT_list[0:20],docs[0:20]):
    net.predict([sdt])
    energies = np.array(activation['visualization_layer']).reshape((-1))
    atoms = mongo.make_atoms_from_doc(doc)
    print(doc['results']['energy'])
    #energies=energies*(1-atoms.get_tags())+atoms.get_tags()*np.mean(energies)
    atoms.set_initial_charges(energies)
    atoms.write('%d.traj'%(index))
    index+=1

In [ ]:
activation['visualization_layer']